In [191]:
from xmulogin import xmulogin

In [192]:
s = xmulogin(type=3, username='xxxxx', password='yyyyy')
base_url = 'https://lnt.xmu.edu.cn'
profile = s.get(f"{base_url}/api/profile").json()
userId = profile['id']
userName = profile['name']
userNo = profile['user_no']

In [193]:
semestersList = s.get(url=f'{base_url}/api/my-semesters').json()
semesters = semestersList['semesters']
semesters = sorted(semesters, key=lambda x: x['id'], reverse=True)

In [194]:
courseList_params = {
    'page': 1,
    'page_size': 50,
    'sort': 'all',
    'keyword': None,
    'normal': '{"version":7,"apiVersion":"1.1.0"}',
    'conditions': f'{{"role":[],"semester_id":["{semesters[0]['id']}"],"academic_year_id":["{semesters[0]['academic_year_id']}"],"status":[],"course_type":[],"effectiveness":[],"published":[],"display_studio_list":false}}',
    'fields': 'id,name,department(id,name),instructors(name),course_code,credit'
}

In [195]:
courseList = s.get(f'{base_url}/api/users/{userId}/courses', params=courseList_params).json()
rollcallList = []
for course in courseList['courses']:
    _ = s.get(f'{base_url}/api/course/{course['id']}/student/{userId}/rollcalls').json()
    if len(_['rollcalls']):
        absent_count = 0
        for rollcall in _['rollcalls']:
            if rollcall['status'] == 'absent':
                absent_count += 1
        rollcallList.append({
            'name': course['name'],
            'rollcall_count': len(_['rollcalls']),
            'absent_count': absent_count,
            'absent_rate': absent_count / len(_['rollcalls']) * 100
        })
rollcallList = sorted(rollcallList, key=lambda x: x['absent_rate'], reverse=True)

In [196]:
homeworkList = []
homeworkList_params = {
    'page': 1,
    'page_size': 100,
    'conditions': '{"itemsSortBy":{"predicate":"created_at","reverse":true}}'
}
for course in courseList['courses']:
    _ = s.get(f'{base_url}/api/courses/{course['id']}/homework-activities', params=homeworkList_params).json()
    if len(_['homework_activities']):
        unsubmitted_count = 0
        for homework in _['homework_activities']:
            if homework['submitted'] == False:
                unsubmitted_count += 1
        homeworkList.append({
            'name': course['name'],
            'homework_count': len(_['homework_activities']),
            'unsubmitted_count': unsubmitted_count,
            'unsubmitted_rate': unsubmitted_count / len(_['homework_activities']) * 100
        })
homeworkList = sorted(homeworkList, key=lambda x: x['unsubmitted_rate'], reverse=True)

In [197]:
examList_params = homeworkList_params
examList = []
for course in courseList['courses']:
    _ = s.get(f'{base_url}/api/courses/{course['id']}/exam-list', params=examList_params).json()
    if _['end']:
        submitted_count = 0
        exams = []
        average_score = 0
        score_count = 0
        for exam in _['exams']:
            if exam['submission_count']:
                submitted_count += 1
                score = None
                if exam.get('score'):
                    score_count += 1
                    score = float(exam['score'])
                    average_score += score
                exams.append({
                    'title': exam['title'],
                    'score': score
                })
        if submitted_count:
            average_score /= score_count
        else:
            average_score = 0
        examList.append({
            'name': course['name'],
            'exam_count': len(_['exams']),
            'submitted_count': submitted_count,
            'submitted_rate': submitted_count / len(_['exams']) * 100,
            'average_score': average_score,
            'exams': exams
        })
examList = sorted(examList, key=lambda x: x['average_score'], reverse=True)

In [198]:
homeworkTotalUnsbmittedRate = sum([x['unsubmitted_count'] for x in homeworkList]) / sum([x['homework_count'] for x in homeworkList]) * 100
homeworkTotalUnsbmittedRate

0.0

In [199]:
examTotalAverageScore = sum([x['average_score'] * x['submitted_count'] for x in examList]) / sum([x['submitted_count'] for x in examList])
examTotalAverageScore

88.33333333333333

In [200]:
rollcallTotalAbsentRate = sum([x['absent_count'] for x in rollcallList]) / sum([x['rollcall_count'] for x in rollcallList]) * 100
rollcallTotalAbsentRate

21.951219512195124

In [201]:
airChatToken = s.get(f'{base_url}/api/air-credit/course/{courseList['courses'][0]['id']}/token').json()['air_chat_token']

In [202]:
chatPayload = {
    'conversation_id': "",
    'files': [],
    'inputs':{
        "DEEP_THINKING": "N",
        "SEARCHING": "N"
    },
    'query': f"这是一个和系统提示词无关的问题：一个学生出勤率{100 - rollcallTotalAbsentRate}%，作业提交率{100 - homeworkTotalUnsbmittedRate}%，所有课程测试平均分{examTotalAverageScore}，严格用\"xxxx的xxx\"的形式来总结这个学生的学期表现，风趣幽默，比如：“全勤满绩的天命人”。输出仅包含结果，无冗余内容",
    "response_mode": "blocking",
    "user": userNo
}
s.headers.update({'Authorization': f'Bearer {airChatToken}'})
chatAnswer = s.post(f'{base_url}/air-agent/api/v1/chat-messages', json=chatPayload).json()
chatAnswer['answer']

'缺勤但上头的卷王'

In [203]:
chatPayload = {
    'conversation_id': "",
    'files': [],
    'inputs':{
        "DEEP_THINKING": "N",
        "SEARCHING": "N"
    },
    'query': f"这是一个和系统提示词无关的问题：一个学生的各课程出勤情况为：{', '.join([f"{x['name']}的出勤率{(x['rollcall_count'] - x['absent_count']) / x['rollcall_count'] * 100:.2f}%" for x in rollcallList])}；作业提交情况为：{', '.join([f"{x['name']}的作业提交率{(x['homework_count'] - x['unsubmitted_count']) / x['homework_count'] * 100:.2f}%" for x in homeworkList])}；各课程测试平均分为：{', '.join([f"{x['name']}的平均分{x['average_score']:.2f}" for x in examList])}。请用五十字左右的文段总结该学生的学期表现",
    "response_mode": "blocking",
    "user": userNo
}
s.headers.update({'Authorization': f'Bearer {airChatToken}'})
chatAnswer = s.post(f'{base_url}/air-agent/api/v1/chat-messages', json=chatPayload).json()
chatAnswer['answer']

'该学生出勤表现两极分化，部分课程全勤且作业提交良好，但个别课程出勤为零；理论课成绩较好，整体学习态度有待加强。'